In [1]:
from sympy import *
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
init_printing(use_latex='mathjax')

# Asymptotic Expansion of Integrals

We'll consider the following integral from Euler $$I(x) = \int_0^\infty \frac{e^{-t}}{1+xt} dt, x\geq 0.$$ 

This first batch of code will expand $(1+xt)^{-1},$ then perform the integration to produce a divergent series. You can compare the accuracy of the approximation to the integral itself with different values of `x_0,trunc`.

In [2]:
x_0 = 0.1
trunc = 5
x,t  = symbols('x t')
Euler_Integral = lambda y: N(integrate(exp(-t)/(1+x*t),(t,0,oo)).subs({x:y}))
display(Euler_Integral(x_0))

J = integrate(exp(-t)*series(1/(1+x*t),t,x0=0,n=trunc),(t,0,oo))
display(J)
display(J.subs({x:x_0}))

0.915633339397881

    4      3      2        
24⋅x  - 6⋅x  + 2⋅x  - x + 1

0.916400000000000

## Perturbed Gaussian Integrals

Most of these methods are built upon the behavior of the Gaussian integral $\int_{-\infty}^\infty e^{-ax^2} dx = \frac1{\sqrt{2\pi a}},$ so we'll begin with a basic perturbation $\int_{-\infty}^\infty e^{-ax^2 - \epsilon x^4} dx$

In [3]:
a,x,epsilon = symbols('a x epsilon')
a_act, eps_act = 1, 0.1
trunc = 4
#subs should happen first because sympy seems to do something with epsilon, 
#causing problems when epsilon is 0
Gaussian_Integral = N(integrate(exp(-a*x**2-epsilon*x**4).subs({a:a_act,epsilon:eps_act}),(x,-oo,oo)))
display(Gaussian_Integral)

dropO = lambda s: sympify(str(s).replace('O','0 *'))
J = exp(-a*x**2)*series(exp(-epsilon*x**4),epsilon,x0=0,n=trunc)
display(J)
J = N(integrate(dropO(J).subs({a:a_act,epsilon:eps_act}),(x,-oo,oo)))
display(J)

1.67408585556458

⎛            2  8    3  12        ⎞      2
⎜       4   ε ⋅x    ε ⋅x      ⎛ 4⎞⎟  -a⋅x 
⎜1 - ε⋅x  + ───── - ────── + O⎝ε ⎠⎟⋅ℯ     
⎝             2       6           ⎠       

1.64969757443460

## Method of Stationary Phase

A variation on this Gaussian idea is done with the Fresnel integral $$I(\epsilon) = \int_{-\infty}^\infty e^{it^2/\epsilon} dt = e^{\text{sgn}(\epsilon)i\pi/4}\sqrt{2\pi|\epsilon|}$$

From this we consider the general integral $$I(\epsilon) = \int_{-\infty}^\infty f(t)e^{i\phi(t)/\epsilon} dt.$$ The key to this analysis are the critical points $c$ where $\phi'(c) = 0.$ 
as $\epsilon\to0,$ the exponential term becomes highly oscillatory, which results in cancellation **except** at the critical points. The expansion is a sum of the following terms at each critiical point
$$I(\epsilon)\sim\int f(c)\exp\frac{i}{\epsilon}\left(\phi(c) + \frac{\phi''(c)}{2}(t-c)^2\right)dt$$ 
$$\sim f(c)e^{i\phi(c)/\epsilon}\int \exp\left[\frac{i\phi''(c)}{2\epsilon} s^2\right]ds$$ 
$$\sim f(c)e^{\text{sgn}(\phi''(c))i\pi/4 + i\phi(c)/\epsilon}\sqrt{\frac{2\pi\epsilon}{|\phi''(c)|}}$$

We can generate the expansion with the following code

In [5]:
t,epsilon = symbols('t epsilon')
f = t**2
phi = -(t**2 - 1)*(t**2 - 4)

def stationary_phase(phi,f,epsilon,t):
    phi_t = phi.diff(t)
    phi_tt = phi.diff(t,t)
    critical_points = [_ for _ in solveset(phi_t,t)]
    expansion = 0
    for c in critical_points:
        a = phi_tt.subs(t,c)
        pc = phi.subs(t,c)
        fc = f.subs(t,c)
        expansion += fc*sqrt(2*pi*epsilon/abs(a))*exp(I*pc/epsilon + sign(a)*I*pi/4)
    return expansion.simplify()
display(stationary_phase(phi,f,epsilon,t))

           ⅈ⋅(-π⋅ε + 9)
           ────────────
               4⋅ε     
√10⋅√π⋅√ε⋅ℯ            
───────────────────────
           2           

## Laplace's Method

If we instead consider the integral $$I(\epsilon) = \int_{-\infty}^\infty f(t)e^{\phi(t)/\epsilon} dt.$$ As before, the terms contributed to the expansion comes out of the critical points. Since the exponent is real, this expansion **only works if there is a unique global maximum of $\phi$.** If located at $x=c,$ we have that $\phi''(c)\leq 0$
$$I(\epsilon)\sim\int f(c)\exp\frac{1}{\epsilon}\left(\phi(c) - \frac{|\phi''(c)|}{2}(t-c)^2\right)dt$$ 
$$\sim f(c)e^{\phi(c)/\epsilon}\int \exp\left[-\frac{|\phi''(c)|}{2\epsilon} s^2\right]ds$$ 
$$\sim f(c)e^{\phi(c)/\epsilon}\sqrt{\frac{2\pi\epsilon}{|\phi''(c)|}}$$

We can generate the expansion with the following code

In [6]:
t,epsilon = symbols('t epsilon')
f = t**2
phi = -(t+1)*(t-1)**2*(t-2)

def laplace_method(phi,f,epsilon,t):
    phi_t = phi.diff(t)
    phi_tt = phi.diff(t,t)
    critical_points = [_ for _ in solveset(phi_t,t)]
    c = critical_points[np.argmax([phi.subs(t,c) for c in critical_points])]
    pc = phi.subs(t,c)
    if limit(phi,t,oo) > pc or limit(phi,t,-oo) > pc:
        raise Exception("Exponential term must have global maximum in domain")
    a = phi_tt.subs(t,c)
    fc = f.subs(t,c)
    expansion = fc*sqrt(2*pi*epsilon/abs(a))*exp(pc/epsilon)
    return expansion.simplify()
display(laplace_method(phi,f,epsilon,t))

                    73⋅√73 + 827
                    ────────────
                       512⋅ε    
√π⋅√ε⋅(49 - 5⋅√73)⋅ℯ            
────────────────────────────────
            ____________        
        8⋅╲╱ 3⋅√73 + 73         

## Method of Steepest Descent

The method of steepest descent approximates complex integrals of the form $$I(\epsilon) = \int_C f(z)e^{\phi(z)/\epsilon} dz$$ on some contour $C,$ which can be changed to some other contour $C'$ since the integrand is analytic. Thus we can deform the contour such that $\text{Im}(\phi)$ is constant, and thus factored out, then $$I(\epsilon) = e^{i\text{Im}(\phi)/\epsilon} = \int_{C'} f(z)e^{\text{Re}(\phi)/\epsilon} dz$$ is solved via Laplace's method.

While an important tool to mention, there is not a nice way to perform such contour operations in SymPy.